In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.pbs_manager import pbs_manager
from autoqchem.helper_classes import pbs_status
from autoqchem.draw_utils import draw


## pbs_manager
### autoQchem job을 관리하는 manager. 누리온 서버에 접속하여 job 생성 및 upload, 삭제 등 job을 관리한다

In [2]:
pm=pbs_manager(user='a1793a01', host='nurion.ksc.re.kr', sftp_host='nurion-dm.ksc.re.kr', workdir='/bo/workspace/nurion')

In [3]:
pm.connect()
## OTP가 필요. password = nuri2024@@

Password(OTP):  ········


AuthenticationException: Authentication failed.

In [10]:
### Set molecule information
### 여러 개의 molecule을 , 로 연결

my_smiles=[
    "c1ccc(F)cc1",
]

In [11]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, num_conf=10))

# Gaussian Job setup
### Gaussian Job 생성 조건

In [12]:
### solvent list : https://gaussian.com/scrf/?tabid=7. 테이블에서 실제 실험 사용할 조건을 선택 기재

for mol in mols:
    pm.create_jobs_for_molecule(mol, light_basis_set='6-31G**', solvent='Ethanol', wall_time='06:00:00')

Molecule InChI=1S/C6H5F/c7-6-4-2-1-3-5-6/h1-5H already exists with the same Gaussian config with tags ['yh_test']. Not creating jobs.


In [13]:
pm.get_job_stats(split_by_can=True)

status,created
can,
C(C1C(C(C(C(O1)O)O)O)O)O,10


## Job 실행
job status가 create 인 job을 실행시키는 작업으로 nurion 서버에 queue가 걸리게 됩니다.\
실제 job을 실행시킬 때 실행하세요.

Job 실행을 위해서는 job file이 넘어갈 수 있도록 pm.connect_sftp() 실행이 필요합니다

In [ ]:
pm.connect_sftp()

## OTP 연결 필요

In [9]:
### job을 nurion에 제출. 제출할 때만 실행시키세요. 

pm.submit_jobs()

# Job 상태 조회
서버에 현재 실행 중이거나 대기 중인 작업을 조회하는 기능입니다. \
job 목록에서 "S"가 status를 의미하며 아래와 같이 상태를 표기합니다 \
Q : Queue 대기 중 \
R : Run 실행 중

In [17]:
pm.retrieve_jobs()  

In [8]:
pm.get_job_stats(split_by_can=True)

status,created
can,
C(C1C(C(C(C(O1)O)O)O)O)O,9


## 결과 DB insert
위의 job이 모두 끝나면 계산 결과를 DB에 넣는 기능입니다. \
현재 노트북에 설치된 mongodb에 넣게 되며 tag를 지정하여 넣으면 추후 해당 tag set을 조회하는데 유용합니다.

In [43]:
### JOB 이 끝난 것을 확인 후 TAG 명을 지정 후 upload. 
pm.upload_done_molecules_to_db(tags=["yh_test"])